In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-oz4d4dp7/unsloth_fb2e9698503c42bc9843e52d036d14df
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-oz4d4dp7/unsloth_fb2e9698503c42bc9843e52d036d14df
  Resolved https://github.com/unslothai/unsloth.git to commit 47ffd39abd02338e8a5f226d0f529347fb7e5f89
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import TensorDataset, DataLoader

import pandas as pd
pd.set_option('display.max_colwidth', None)
from IPython.display import display, HTML

from tqdm.auto import tqdm

from unsloth import FastLanguageModel

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Arguments

#model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit"
model_name = "unsloth/gemma-7b-it-bnb-4bit"
#model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 8192,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [ ]:
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template
tokenizer.padding_side = "right"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

Llama3


In [ ]:
messages = [
    {"from": "system", "value": "Answer the following questions in Italian."},
     {"from": "human", "value": "in which country is Lahijan located?"}
]

inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")
text_streamer = TextStreamer(tokenizer)
outputs = model.generate(input_ids = inputs,pad_token_id=tokenizer.eos_token_id, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)

<bos><|start_header_id|>system<|end_header_id|>

Answer the following questions in Italian.<|eot_id|><|start_header_id|>human<|end_header_id|>

in which country is Lahijan located?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Lahijan is located in Iran.<|eot_id|><|start_header_id|>expert<|end_header_id|>



KeyboardInterrupt: 

Gemma and Mistral

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

rules =  [[
                "the hit song",
                "CausesDesire",
                "clapping"
            ],
            [
                "clapping along with the music",
                "PartOf",
                "supporting the band"
            ],
            [
                "politely listening in silence",
                "Propertyof",
                "audience"
            ],
            [
                "audience",
                "CapableOf",
                "leaving the concert"
            ],
            [
                "audience",
                "CapableOf",
                "being polite"
            ]]

options_str = str({
 '5b9441298f': 'Seasoned salted crackers shaped like oysters',
 '00310c3462': 'Creamy clam chowder made with whole milk and baby clams',
 '8635ea3d3c': 'Tomato mussel soup containing dry white wine',
 '08cb462fdf': 'Simple creamy oyster soup',
 '52b83497d8': 'Warm vegetable soup containing tomatoes, peas, corn, carrots, and potatoes'})

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Task: You will be given a sentence about an event. Also, a number of rules in the form of a Knowledge Base are presented to you. For this event, two possible consequences are given. Onee of these two consequences is more probable and supported by the knowledge base. Dtermine which consequence is the more probable one. Also, You must provide a proof for your answer by using the rules from the Knowledge Base. First, copy the consequence you think can be inferred. Then, in the next line, provide your proof by stating the rules you used from the Knowledge Base. Let's think step by step.", # instruction
        f"Query: The band played their hit song. KB: {str(rules)}. Consequence 1: The audience clapped along to the music. Consequence 2: The audience politely listened in silence.']", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

# messages = [
#     {"from": "human", "value": f"""Task: You will be given a query about an object x. You are a helpful and smart assistant trying to answer this query. In order to do this, a fact about x and a set of rules are provided to you in a Knowledge Base. Using these rules, you must both provide an answer to the query (the answer has to be \"True\" or \"False\") and give a proof of your answer by using the rules from Knowledge Base. Think step-by-step and try to use the rules one-by-one to answer the query. Begin your response by providing the proof and stating the rules you used from the knowledge base to give the answer. Then, give your final answer to the query by saying either \"Therefore, the answer to the query is True\" or \"Therefore, the answer to the query is False\" and not saying anything else.\
#      Query: x is imaginary. KB: ['all real numbers are not imaginary.', 'x is a real number'] {EOS_TOKEN}"""}

# ]



#inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")
text_streamer = TextStreamer(tokenizer)
#outputs = model.generate(input_ids = inputs,pad_token_id=tokenizer.eos_token_id, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)
outputs = model.generate(**inputs,pad_token_id=tokenizer.eos_token_id, max_new_tokens = 1024, use_cache = True)


In [ ]:
tokenizer.batch_decode(outputs)


["<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nTask: You will be given a sentence about an event. Also, a number of rules in the form of a Knowledge Base are presented to you. For this event, two possible consequences are given. Onee of these two consequences is more probable and supported by the knowledge base. Dtermine which consequence is the more probable one. Also, You must provide a proof for your answer by using the rules from the Knowledge Base. First, copy the consequence you think can be inferred. Then, in the next line, provide your proof by stating the rules you used from the Knowledge Base. Let's think step by step.\n\n### Input:\nQuery: The band played their hit song. KB: [['the hit song', 'CausesDesire', 'clapping'], ['clapping along with the music', 'PartOf', 'supporting the band'], ['politely listening in silence', 'Propertyof', 'au

Loading the Dataset

In [ ]:
##Importing the data file from google drive
import os, sys
import numpy as np
import json, random, pickle
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


Class for Reading the Data from the dataset

In [ ]:
# custom logger to save LLM responses and resolution processes
class logger:
    def __init__(self, experiment_name):
        self.experiment_name = experiment_name

    def __call__(self, variable_name, variable_value):
        with open(f'{self.experiment_name}_records.txt', 'a') as file:
            file.write(f"{variable_name}: {variable_value}\n")


In [ ]:
random.seed(98)
np.random.seed(98)

class Dataset:
  def __init__(self, dataset_name, misleading_rules_no, masked_rules_no, log):
    self.dataset_name = dataset_name
    self.misleading_rules_no = misleading_rules_no
    self.masked_rules_no = masked_rules_no
    self.data = self._load_dataset()
    self.len = len(self.data)
    self.dataset_kb = self._load_kb(dataset_name)
    self.log = log

  def __call__(self, row_number):
    row = self.data[row_number]
    if self.dataset_name == 'ProntoQA':
      query, rules, answer, aspects = row['query'], row['context'], row['answer'], None
      self.log("Query: ", query); self.log("Answer: ", answer); self.log("Rules: ", rules)
      options = rules
    elif self.dataset_name == 'COPA':
      query, options, answer, aspects = row['options'], row['rules'], row['answer'], row['query']

    elif self.dataset_name == 'Recipe-MPR':
      query, options, answer, aspects = row['query'], row['options'], row['answer'], list(row['correctness_explanation'].keys())

    return query, options, answer, aspects

  def _perturb_rules(self, rules_fol, dataset_kb, masked_rules_no, misleading_rules_no):
    universal_rules = rules_fol[:-1]
    masked_rules = random.sample(universal_rules, masked_rules_no)
    unmasked_rules = [rule for rule in rules_fol if rule not in masked_rules]
    random.shuffle(dataset_kb)
    misleading_rules = []
    for rule in dataset_kb:
        if len(misleading_rules) == misleading_rules_no:
            break
        if str(rule) not in [str(x) for x in masked_rules]:
            misleading_rules.append(rule)

    perturbed_rules = [] + unmasked_rules
    for rule in misleading_rules:
        if str(rule) not in [str(x) for x in perturbed_rules]:
            perturbed_rules.append(rule)
    return perturbed_rules

  def _load_dataset(self):

    if self.dataset_name == 'ProntoQA':
          with open(os.path.join(self.dataset_name + f'_list_context_misleading{self.misleading_rules_no}_masked{self.masked_rules_no}.json') , "r") as f:
                    data = json.load(f)
    elif self.dataset_name == 'COPA':
      with open(os.path.join(self.dataset_name + '.json') , "r") as f:
                    data = json.load(f)

    elif self.dataset_name == 'Recipe-MPR':
        with open(os.path.join(self.dataset_name + '.json') , "r") as f:
            data = json.load(f)
    return data
  def _load_kb(self, dataset_name):

    if dataset_name == 'ProntoQA':
        with open('ProntoQA_KB_NL.json' , "r") as f:
            kb = json.load(f)
    else:
        kb = None
    return kb

In [ ]:
dataset_name = 'Recipe-MPR'
llm_name = 'gemma'
#llm_name = 'mistral'
os.chdir(f'/content/drive/My Drive/AW/{dataset_name}')
masked_rules_no = 0
misleading_rules_no = 0
experiment_name = f'{dataset_name}_{llm_name}_masked{masked_rules_no}_misleading_{misleading_rules_no}'
log = logger(experiment_name)

dataset = Dataset(dataset_name, misleading_rules_no, masked_rules_no, log)

In [ ]:
query, options, answer, aspects = dataset(0)

In [ ]:
query

['The audience clapped along to the music.',
 'The audience politely listened in silence.']

In [ ]:
options

[['the hit song', 'CausesDesire', 'clapping'],
 ['clapping along with the music', 'PartOf', 'supporting the band'],
 ['politely listening in silence', 'Propertyof', 'audience'],
 ['audience', 'CapableOf', 'leaving the concert'],
 ['audience', 'CapableOf', 'being polite']]

In [ ]:
answer

'The audience clapped along to the music.'

In [ ]:
aspects

'The band played their hit song.'

Function for forming the prompt from the data instance

In [ ]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

def form_prompt(model_name, dataset_name, query, options, aspects):

    if dataset_name == 'Recipe-MPR':
      system_prompt = """Pick the option that best satisfies the query and concisely explain your answer. You must answer with: "Option <option number>. <justification>" and nothing else."""
      user_prompt = f"""Query: {query}
      Options: {str(options)}"""

    elif dataset_name == 'COPA':
       cons1, cons2 = query[0], query[1]
       system_prompt = """Task: You will be given a sentence about an event. Also, a number of rules in the form of a Knowledge Base are presented to you. For this event, two possible consequences are given. You need to determine which of these consequences can be inferred from the event and the rules in the Knowledge Base. You must provide a proof for your answer by using the rules from the Knowledge Base. First, copy the consequence you think can be inferred. Then, in the next line, provide your proof by stating the rules you used from the Knowledge Base. Let's think step by step."""
       user_prompt = f"""Event: { aspects }
       KB: { options }
       Consequence1: { cons1 }
       Consequence2: { cons2 }"""


    elif dataset_name == 'ProntoQA':

      system_prompt = """Task: You will be given a query about an object x. You are a helpful and smart assistant trying to answer this query. In order to do this, a fact about x and a set of rules are provided to you in a Knowledge Base. Using these rules, you must both provide an answer to the query (the answer has to be "True" or "False") and give a proof of your answer by using the rules from Knowledge Base. Think step-by-step and try to use the rules one-by-one to answer the query. Begin your response by providing the proof and stating the rules you used from the knowledge base to give the answer. Then, give your final answer to the query by saying either "Therefore, the answer to the query is True" or "Therefore, the answer to the query is False" and not saying anything else."""
      user_prompt = f"""Query: {query}
      KB: {options}"""
    if 'llama' in model_name:
      messages = [{"from": "system", "value": system_prompt}, {"from": "human", "value": user_prompt}]
      inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")
    elif 'gemma' in model_name or 'mistral':
      alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

      ### Instruction:
      {}

      ### Input:
      {}

      ### Response:
      {}"""
      EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
      if dataset_name == 'COPA':
        cons1, cons2 = query[0], query[1]

        inputs = tokenizer(
        [
          alpaca_prompt.format(
          "Task: You will be given a sentence about an event. Also, a number of rules in the form of a Knowledge Base are presented to you. For this event, two possible consequences are given. Onee of these two consequences is more probable and supported by the knowledge base. Dtermine which consequence is the more probable one. Also, You must provide a proof for your answer by using the rules from the Knowledge Base. First, copy the consequence you think can be inferred. Then, in the next line, provide your proof by stating the rules you used from the Knowledge Base. Let's think step by step.", # instruction
          f"Event: {aspects} Knowledge Base: {options} Consequence 1: {cons1} Consequence2: {cons2}", # input
          "", # output - leave this blank for generation!
          )
          ], return_tensors = "pt").to("cuda")

      if dataset_name == 'Recipe-MPR':
        inputs = tokenizer(
        [
          alpaca_prompt.format(
          "Task: Consider the provided query and the set of options. You must pick the option that is most suitable for the query. Think step by step. First, explain your reason for why you think this recipe is the most proper. Remember that you have to state the reason first. Then, mention the most proper recipe by saying: Therefore, the selected option is: <option number>.", # instruction
          f"Query: {query}. Options: {str(options)}", # input
          "", # output - leave this blank for generation!
          )
          ], return_tensors = "pt").to("cuda")

      if dataset_name == 'ProntoQA':
        inputs = tokenizer(
        [
          alpaca_prompt.format(
          "Task: You will be given a query about an object x. You are a helpful and smart assistant trying to answer this query. In order to do this, a fact about x and a set of rules are provided to you in a Knowledge Base. Using these rules, you must both provide an answer to the query (the answer has to be \"True\" or \"False\") and give a proof of your answer by using the rules from Knowledge Base. Think step-by-step and try to use the rules one-by-one to answer the query. Begin your response by providing the proof and stating the rules you used from the knowledge base to give the answer. Then, give your final answer to the query by saying either \"Therefore, the answer to the query is True\" or \"Therefore, the answer to the query is False\" and not saying anything else.", # instruction
          f"Query: {query}. KB: {options}", # input
          "", # output - leave this blank for generation!
          )
          ], return_tensors = "pt").to("cuda")


    return inputs

Function to process the final answer from the model output.

In [ ]:
def process_output(dataset_name, output_string):

  if dataset_name == 'COPA':
    final_sentence = str(output_string[0])
    return final_sentence

  if dataset_name == 'Recipe-MPR':
    final_sentence = str(output_string[0].split('Therefore')[1:]).split('Note:')[0]
    return final_sentence

  if dataset_name == 'ProntoQA':
    #final_sentence = output_string[0].split('.')[-2:]
    final_sentence = str(output_string[0].split('Therefore')[3:]).split('Note:')[0]

    if 'true' in final_sentence.lower():
      return 'true'
    elif 'false' in final_sentence.lower():
      return 'false'
    else:
      return None

Main loop for running the model and evaluating.

In [ ]:
from tqdm import tqdm
outcomes = []

#for i in tqdm(range(dataset.len)):
for i in tqdm(range(1)):
  query, options, answer, aspects = dataset(i)

  if dataset_name == 'Recipe-MPR':
    new_options = {}
    number2id = {}
    for i, id in enumerate(options):
      number2id[i] = id
      new_options[i] = options[id]
    options = new_options
    answer = '0'


  log("Query:", query); log("Answer:", answer); log("Options:", options)
  inputs = form_prompt(model_name, dataset_name, query, options, aspects)

  #inputs = tokenizer.apply_chat_template(test_message, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")
  #outputs = model.generate(input_ids = inputs,pad_token_id=tokenizer.eos_token_id, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)
  if 'llama' in model_name:
    outputs = model.generate(input_ids = inputs,pad_token_id=tokenizer.eos_token_id, max_new_tokens = 1024, use_cache = True)
  elif 'gemma' in model_name or 'mistral':
    outputs = model.generate(**inputs,pad_token_id=tokenizer.eos_token_id, max_new_tokens = 512, use_cache = True)

  output_string = tokenizer.batch_decode(outputs)
  print(output_string)
  log("output string", output_string)
  selected_option = process_output(dataset_name,output_string)
  print(selected_option)
  print(str(answer).lower())
  log("Selected option:", selected_option)
  correctness = str(answer).lower() in str(selected_option).lower()
  outcomes.append(correctness)
  if not correctness:
    log("Incorrect Answer:", answer)

  print("Accuracy so far: ", np.mean(outcomes))


print(f"Accuracy: {np.mean(outcomes)}"); print(f"Correct: {np.sum(outcomes)}"); print(f"Incorrect: {np.sum(np.logical_not(outcomes))}")
log("Accuracy: ", np.mean(outcomes)); log("Correct: ", np.sum(outcomes)); log("Incorrect: ", np.sum(np.logical_not(outcomes)))



100%|██████████| 1/1 [00:34<00:00, 34.08s/it]

["<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n      ### Instruction:\n      Task: Consider the provided query and the set of options. You must pick the option that is most suitable for the query. First, explain your reason for why you think this recipe is the most proper. Remember that you have to state the reason first. Then, mention the most proper recipe by saying: Therefore, the selected option is: <option number>.\n\n      ### Input:\n      Query: I want to make a warm dish containing oysters. Options: {0: 'Simple creamy oyster soup', 1: 'Seasoned salted crackers shaped like oysters', 2: 'Creamy clam chowder made with whole milk and baby clams', 3: 'Tomato mussel soup containing dry white wine', 4: 'Warm vegetable soup containing tomatoes, peas, corn, carrots, and potatoes'}\n\n      ### Response:\n      The selected option is: 0<pad><pad><pad><pad><pad><pad><p